In this notebook we will make a multi-agent web browser: an agentic system with several agents collaborating to solve problems using the web!

```md
              +----------------+
              | Manager agent  |
              +----------------+
                       |
        _______________|______________
       |                              |
  Code interpreter   +--------------------------------+
       tool          |         Managed agent          |
                     |      +------------------+      |
                     |      | Web Search agent |      |
                     |      +------------------+      |
                     |         |            |         |
                     |  Web Search tool     |         |
                     |             Visit webpage tool |
                     +--------------------------------+
```

⚡️ Our agent will be powered by meta-llama/Meta-Llama-3.1-70B-Instruct using HfApiEngine class that uses HF’s Inference API: the Inference API allows to quickly and easily run any OS model.



In [1]:
!pip install -q markdownify duckduckgo-search "transformers[agents]" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00


In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [12]:
model = "Qwen/Qwen2.5-72B-Instruct"

create web search tool (web browsing)

In [13]:
import re
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
from transformers.agents import tool

@tool
def visit_webpage(url: str) -> str:
  """ visit webpage of url and return content in md string
  Args:
        url: The URL of the webpage to visit.
  Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
  """
  try:
    resp = requests.get(url)
    resp.raise_for_status() # exception for bad status codes

    # html to md
    md_content = md(resp.text).strip()

    # remove line breaks
    md_content = re.sub(r"\n{3,}", "\n\n", md_content)

    return md_content

  except RequestException as e:
    return f"error fetching page: {str(e)}"
  except Exception as e:
    return f"unexpected error: {str(e)}"


print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

Hugging Face \- Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu
move to sidebar
hide

 Navigation
 

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Co


## build multi agent system

Which configuration to choose for this agent?

Web browsing is a single-timeline task that does not require parallel tool calls, so JSON tool calling works well for that. We thus choose a ReactJsonAgent.
Also, since sometimes web search requires exploring many pages before finding the correct answer, we prefer to increase the number of max_iterations to 10.

In [14]:
from transformers.agents import (
    ReactCodeAgent,
    ReactJsonAgent,
    HfApiEngine,
    ManagedAgent
)
from transformers.agents.search import DuckDuckGoSearchTool

llm_engine = HfApiEngine(model)

web_agent = ReactJsonAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    llm_engine=llm_engine,
    max_iterations=10
)

# wrap to make it callable by its manager agent
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search",
    description="run web searches, give query as arg"
)

creating manager agent

Since this agent is the one tasked with the planning and thinking, advanced reasoning will be beneficial, so a ReactCodeAgent will be the best choice.

Since this agent is the one tasked with the planning and thinking, advanced reasoning will be beneficial, so a ReactCodeAgent will be the best choice.



In [15]:
manager_agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    managed_agents=[managed_web_agent],
    additional_authorized_imports=["time", "datetime"],
)

In [16]:
manager_agent.run("How many years ago was Stripe founded?")

======== New task ========
How many years ago was Stripe founded?
=== Agent thoughts:
Thought: To find out how many years ago Stripe was founded, I need to first determine the year Stripe was founded. I can use the `search` tool to find this information.
>>> Agent is executing the code below:
stripe_founded = search(query="When was Stripe founded?")
print("Stripe was founded:", stripe_founded)
====
Code execution failed due to the following error:
ManagedAgent.__call__() missing 1 required positional argument: 'request'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/agents/agents.py", line 1135, in step
    result = self.python_evaluator(
  File "/usr/local/lib/python3.10/dist-packages/transformers/agents/python_interpreter.py", line 895, in evaluate_python_code
    result = evaluate_ast(node, state, static_tools, custom_tools, authorized_imports)
  File "/usr/local/lib/python3.10/dist-packages/transformers/agents/python_interpreter.py",

14

💡 You can easily extend this to more agents: one does the code execution, one the web search, one handles file loadings…

🤔💭 One could even think of doing more complex, tree-like hierarchies, with one CEO agent handling multiple middle managers, each with several reports.